<a href="https://colab.research.google.com/github/AugustoGuimaraesUFRJ/Algoritmo-Grafos/blob/main/TCC_C%C3%B3digo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

augustoguimaraes_bertimbau_tcc_model_path = kagglehub.dataset_download('augustoguimaraes/bertimbau-tcc-model')
augustoguimaraes_miniml_path = kagglehub.dataset_download('augustoguimaraes/miniml')
augustoguimaraes_treinamento_e_teste_path = kagglehub.dataset_download('augustoguimaraes/treinamento-e-teste')
augustoguimaraes_treino_mlm_path = kagglehub.dataset_download('augustoguimaraes/treino-mlm')
augustoguimaraes_treinamentoad_path = kagglehub.dataset_download('augustoguimaraes/treinamentoad')

print('Data source import complete.')


In [ ]:
import pandas as pd
import torch
import random
import re
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import (
    BertTokenizer, BertForMaskedLM,
    AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
)
from datasets import Dataset
from sentence_transformers import SentenceTransformer, util
import os
import unicodedata
import string
from sklearn.model_selection import train_test_split
os.environ["WANDB_DISABLED"] = "true"
import matplotlib.pyplot as plt

import logging

# Desativa qualquer barra de progresso do tqdm e logs do transformers
tqdm.disable = True
logging.getLogger("transformers").setLevel(logging.ERROR)
logging.getLogger("sentence_transformers").setLevel(logging.ERROR)

In [ ]:
model_name = "/kaggle/input/bertimbau-tcc-model/bert-base-portuguese-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForMaskedLM.from_pretrained(model_name)
model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"✅ Modelo carregado no dispositivo: {device}")

✅ Modelo carregado no dispositivo: cuda


In [ ]:
# Caminhos dos arquivos
train_path = "/kaggle/input/treinamento-e-teste/train_df.csv"
test_path = "/kaggle/input/treinamento-e-teste/test_df.csv"

# Carregar datasets
train_final = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

#  Renomear 'class' → 'labels'
train_final = train_final.rename(columns={"class": "labels"})
test_df = test_df.rename(columns={"class": "labels"})

#  Remover nulos e garantir tipo string
for name, df_ in {"train": train_final, "test": test_df}.items():
    df_.dropna(subset=["text", "labels"], inplace=True)
    df_["text"] = df_["text"].astype(str)
    df_.loc[df_["text"].str.strip() == "", "text"] = np.nan
    df_.dropna(subset=["text"], inplace=True)
    print(f" {name}: {len(df_)} linhas")

#  Tokenização
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(p.label_ids, preds, average="binary")
    acc = accuracy_score(p.label_ids, preds)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

train_dataset = Dataset.from_pandas(train_final)
test_dataset = Dataset.from_pandas(test_df)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# ============================================================
# 9 Treinamento e avaliação
# ============================================================
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="no",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()
metrics = trainer.evaluate()
print(metrics)

# ============================================================
# 9. Análise de Erros
# ============================================================
preds_output = trainer.predict(test_dataset)
preds = np.argmax(preds_output.predictions, axis=1)

result_df = test_df.copy()
result_df["preds"] = preds

total = len(result_df)
acertos = (result_df["labels"] == result_df["preds"]).sum()
erros = total - acertos
fp = ((result_df["preds"] == 1) & (result_df["labels"] == 0)).sum()
fn = ((result_df["preds"] == 0) & (result_df["labels"] == 1)).sum()

print(f"\n=== RESULTADOS DE ERRO ===")
print(f"Total: {total}")
print(f" Acertos: {acertos}")
print(f" Erros: {erros}")
print(f"  ↳ Falsos Positivos (prevê ofensa mas não é): {fp}")
print(f"  ↳ Falsos Negativos (não detectou ofensa real): {fn}")

# Mostrar exemplos
falsos_positivos = result_df[(result_df["preds"] == 1) & (result_df["labels"] == 0)]
falsos_negativos = result_df[(result_df["preds"] == 0) & (result_df["labels"] == 1)]

print("\n===  FALSOS POSITIVOS ===")
for t in falsos_positivos["text"].head(30):
    print("•", t)

print("\n===  FALSOS NEGATIVOS ===")
for t in falsos_negativos["text"].head(30):
    print("•", t)

 train: 5322 linhas
 test: 1331 linhas


Map:   0%|          | 0/5322 [00:00<?, ? examples/s]

Map:   0%|          | 0/1331 [00:00<?, ? examples/s]

/tmp/ipykernel_37/2487770449.py:63: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


{'loss': 0.5415, 'grad_norm': 16.501218795776367, 'learning_rate': 1.705705705705706e-05, 'epoch': 0.15015015015015015}
{'loss': 0.3196, 'grad_norm': 8.104938507080078, 'learning_rate': 1.4054054054054055e-05, 'epoch': 0.3003003003003003}
{'loss': 0.2922, 'grad_norm': 8.398772239685059, 'learning_rate': 1.1051051051051051e-05, 'epoch': 0.45045045045045046}
{'loss': 0.273, 'grad_norm': 2.8173561096191406, 'learning_rate': 8.048048048048048e-06, 'epoch': 0.6006006006006006}
{'loss': 0.2734, 'grad_norm': 21.57377052307129, 'learning_rate': 5.045045045045045e-06, 'epoch': 0.7507507507507507}
{'loss': 0.2606, 'grad_norm': 2.817598581314087, 'learning_rate': 2.0420420420420424e-06, 'epoch': 0.9009009009009009}
{'eval_loss': 0.23450876772403717, 'eval_accuracy': 0.9120961682945155, 'eval_precision': 0.9159159159159159, 'eval_recall': 0.9090909090909091, 'eval_f1': 0.912490650710546, 'eval_runtime': 6.9917, 'eval_samples_per_second': 190.369, 'eval_steps_per_second': 12.014, 'epoch': 1.0}
{'tr

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


{'eval_loss': 0.23450876772403717, 'eval_accuracy': 0.9120961682945155, 'eval_precision': 0.9159159159159159, 'eval_recall': 0.9090909090909091, 'eval_f1': 0.912490650710546, 'eval_runtime': 7.3556, 'eval_samples_per_second': 180.951, 'eval_steps_per_second': 11.42, 'epoch': 1.0}
{'eval_loss': 0.23450876772403717, 'eval_accuracy': 0.9120961682945155, 'eval_precision': 0.9159159159159159, 'eval_recall': 0.9090909090909091, 'eval_f1': 0.912490650710546, 'eval_runtime': 7.3556, 'eval_samples_per_second': 180.951, 'eval_steps_per_second': 11.42, 'epoch': 1.0}


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(



=== RESULTADOS DE ERRO ===
Total: 1331
 Acertos: 1214
 Erros: 117
  ↳ Falsos Positivos (prevê ofensa mas não é): 56
  ↳ Falsos Negativos (não detectou ofensa real): 61

===  FALSOS POSITIVOS ===
• Cada dia se queimando mais
• É a treva.
• SENSACIONAL É BEM PÔR AI MESMOOO, ELE ESTÁ PREOCUPADO PORQUÊ SERA, QUEM Ñ DEVE Ñ TEME, SIMPLES ASSIM!!!
• O Brasil lutando para sair da Grota e alguns preocupado com a tal de Greta. Vamos ficar até quando nessa?
• Dupla imbatível nas argumentações inteligentes contra esse governo desgovernado que estamos pagando....aff
• E nós que podemos ter perdido uma Greta Thunberg cruelmente assasinada por witizelassassino witizelgenocida. Ághata nossa Menina Maravilha tinha apenas 8 anos seu discurso era apenas a boneca Mônica, mas quem poderá afirmar ao contrário que aos 16 anos trocaria Mônica pela bandeira de um mundo mais justo e sustentável.
• Nem era pra ter sido preso! LulaInocente
• Uma menina chega as capas aos 16 anos e não são em páginas policiais,mu

In [ ]:
train_path = "/kaggle/input/treinamento-e-teste/train_df.csv"
train_df = pd.read_csv(train_path)

train_df = train_df[train_df['text'].apply(lambda x: isinstance(x, str) and x.strip() != '')]
print("📊 Linhas totais:", len(train_df))
print(train_df.info())
print("\nAmostra de tipos incorretos:")
print(train_df[~train_df['text'].apply(lambda x: isinstance(x, str))].head())

📊 Linhas totais: 5322
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5322 entries, 0 to 5321
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    5322 non-null   object
 1   class   5322 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 83.3+ KB
None

Amostra de tipos incorretos:
Empty DataFrame
Columns: [text, class]
Index: []


In [ ]:
model_name = "/kaggle/input/bertimbau-tcc-model/bert-base-portuguese-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForMaskedLM.from_pretrained(model_name)
model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"✅ Modelo carregado no dispositivo: {device}")

def mlm_augment(text, idx=None, prob=0.15, n_aug=2):
    """
    Gera frases aumentadas via Masked Language Modeling (BERTimbau)
    e associa cada uma ao índice original do texto.

    Args:
        text (str): Texto original.
        idx (int, optional): Índice da frase original no dataset.
        prob (float): Probabilidade de mascarar um token.
        n_aug (int): Quantas versões aumentadas gerar.

    Returns:
        list[dict]: Lista com {"orig_idx", "text_original", "text_augmentada"}
    """
    if not isinstance(text, str) or text.strip() == "":
        return []

    tokens = tokenizer.tokenize(text)
    augmented_samples = []

    for _ in range(n_aug):
        masked_tokens = tokens.copy()
        for i in range(len(masked_tokens)):
            if random.random() < prob and masked_tokens[i].isalpha():
                masked_tokens[i] = tokenizer.mask_token

        masked_text = tokenizer.convert_tokens_to_string(masked_tokens)
        inputs = tokenizer(masked_text, return_tensors='pt').to(device)

        with torch.no_grad():
            outputs = model(**inputs)
            predictions = outputs.logits

        predicted_indices = torch.argmax(predictions, dim=-1)
        if predicted_indices.dim() == 0:
            predicted_indices = predicted_indices.unsqueeze(0)

        predicted_tokens = tokenizer.convert_ids_to_tokens(predicted_indices[0].tolist())

        result_tokens = [
            predicted_tokens[i] if masked_tokens[i] == tokenizer.mask_token else masked_tokens[i]
            for i in range(len(masked_tokens))
        ]

        new_text = tokenizer.convert_tokens_to_string(result_tokens)
        augmented_samples.append({
            "orig_idx": idx,
            "text_original": text,
            "text_augmentada": new_text
        })

    return augmented_samples


# ============================================================
# Carregar dataset
# ============================================================
train_path = "/kaggle/input/treinamento-e-teste/train_df.csv"
train_df = pd.read_csv(train_path)

augmented_rows = []

# Itera sobre cada linha do treino
for idx, row in tqdm(train_df.iterrows(), total=len(train_df)):
    # Gera novas versões da frase
    new_samples = mlm_augment(
        text=row["text"],
        idx=idx,           # índice original da linha no treino
        prob=0.15,         # 15% das palavras mascaradas
        n_aug=2            # duas versões aumentadas por frase
    )

    # Adiciona resultados ao dataset final
    for sample in new_samples:
        new_text = sample["text_augmentada"]

        augmented_rows.append({
            "orig_idx": sample["orig_idx"],
            "text_original": sample["text_original"],
            "text_augmentada": new_text,
            "class": row["class"]
        })

# Cria DataFrame com os textos aumentados
df_aug = pd.DataFrame(augmented_rows)

✅ Modelo carregado no dispositivo: cuda


100%|██████████| 5322/5322 [01:33<00:00, 56.81it/s]


In [ ]:
df_aug= df_aug[["text_augmentada", "class"]].rename(columns={"text_augmentada": "text"})

print(df_aug.shape)
df_aug.head()

(10644, 2)


,text,class
0,Parabéns meu presidente,0
1,Parabéns meu meu,0
2,Valeuu analise .,0
3,Valeu seu seue .,0
4,PT roubando o o povo ! !,1


In [ ]:
train_df = pd.read_csv(train_path)
train_df.head()

,text,class
0,Parabéns meu presidente,0
1,Valeu seu analise.,0
2,PT roubando o país inteiro!!,1
3,Lula livre.,0
4,Ele esqueceu de amadurecer,0


In [ ]:
train_final = pd.concat([train_df, df_aug], ignore_index=True)
train_final.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,text,labels,class
0,Parabéns meu presidente,0.0,NaN
1,Valeu seu analise.,0.0,NaN
2,PT roubando o país inteiro!!,1.0,NaN
3,Lula livre.,0.0,NaN
4,Ele esqueceu de amadurecer,0.0,NaN


In [ ]:
train_path = "/kaggle/input/treinamento-e-teste/train_df.csv"
test_path = "/kaggle/input/treinamento-e-teste/test_df.csv"

# Carregar datasets
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

train_final = pd.concat([train_df, df_aug], ignore_index=True)

#  Renomear 'class' → 'labels'
train_final = train_final.rename(columns={"class": "labels"})
test_df = test_df.rename(columns={"class": "labels"})


train_final = train_final[~train_final["text"].str.contains(r"\[UNK\]", regex=True, na=False)]
train_final = train_final.drop_duplicates(subset=["text"], keep="first").reset_index(drop=True)

#  Remover nulos e garantir tipo string
for name, df_ in {"train": train_final, "test": test_df}.items():
    df_.dropna(subset=["text", "labels"], inplace=True)
    df_["text"] = df_["text"].astype(str)
    df_.loc[df_["text"].str.strip() == "", "text"] = np.nan
    df_.dropna(subset=["text"], inplace=True)
    print(f" {name}: {len(df_)} linhas")

#  Tokenização
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(p.label_ids, preds, average="binary")
    acc = accuracy_score(p.label_ids, preds)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

train_dataset = Dataset.from_pandas(train_final)
test_dataset = Dataset.from_pandas(test_df)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# ============================================================
# 9 Treinamento e avaliação
# ============================================================
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="no",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()
metrics = trainer.evaluate()
print(metrics)

# ============================================================
# 9. Análise de Erros
# ============================================================
preds_output = trainer.predict(test_dataset)
preds = np.argmax(preds_output.predictions, axis=1)

result_df = test_df.copy()
result_df["preds"] = preds

total = len(result_df)
acertos = (result_df["labels"] == result_df["preds"]).sum()
erros = total - acertos
fp = ((result_df["preds"] == 1) & (result_df["labels"] == 0)).sum()
fn = ((result_df["preds"] == 0) & (result_df["labels"] == 1)).sum()

print(f"\n=== RESULTADOS DE ERRO ===")
print(f"Total: {total}")
print(f" Acertos: {acertos}")
print(f" Erros: {erros}")
print(f"  ↳ Falsos Positivos (prevê ofensa mas não é): {fp}")
print(f"  ↳ Falsos Negativos (não detectou ofensa real): {fn}")

# Mostrar exemplos
falsos_positivos = result_df[(result_df["preds"] == 1) & (result_df["labels"] == 0)]
falsos_negativos = result_df[(result_df["preds"] == 0) & (result_df["labels"] == 1)]

print("\n===  FALSOS POSITIVOS ===")
for t in falsos_positivos["text"].head(30):
    print("•", t)

print("\n===  FALSOS NEGATIVOS ===")
for t in falsos_negativos["text"].head(30):
    print("•", t)

 train: 13232 linhas
 test: 1331 linhas


Map:   0%|          | 0/13232 [00:00<?, ? examples/s]

Map:   0%|          | 0/1331 [00:00<?, ? examples/s]

/tmp/ipykernel_37/719385155.py:68: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


{'loss': 0.5433, 'grad_norm': 10.164219856262207, 'learning_rate': 1.8814993954050786e-05, 'epoch': 0.060459492140266025}
{'loss': 0.3297, 'grad_norm': 10.76231575012207, 'learning_rate': 1.760580411124547e-05, 'epoch': 0.12091898428053205}
{'loss': 0.2985, 'grad_norm': 8.488628387451172, 'learning_rate': 1.6396614268440147e-05, 'epoch': 0.18137847642079807}
{'loss': 0.2696, 'grad_norm': 28.817798614501953, 'learning_rate': 1.5187424425634827e-05, 'epoch': 0.2418379685610641}
{'loss': 0.301, 'grad_norm': 6.734292984008789, 'learning_rate': 1.3978234582829506e-05, 'epoch': 0.3022974607013301}
{'loss': 0.2726, 'grad_norm': 7.549954414367676, 'learning_rate': 1.2769044740024186e-05, 'epoch': 0.36275695284159615}
{'loss': 0.2785, 'grad_norm': 14.627341270446777, 'learning_rate': 1.1559854897218865e-05, 'epoch': 0.42321644498186217}
{'loss': 0.2576, 'grad_norm': 8.157594680786133, 'learning_rate': 1.0350665054413544e-05, 'epoch': 0.4836759371221282}
{'loss': 0.2148, 'grad_norm': 4.024790763

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


{'eval_loss': 0.25742781162261963, 'eval_accuracy': 0.9143501126972201, 'eval_precision': 0.908957415565345, 'eval_recall': 0.9225037257824144, 'eval_f1': 0.9156804733727811, 'eval_runtime': 7.2263, 'eval_samples_per_second': 184.188, 'eval_steps_per_second': 11.624, 'epoch': 1.0}
{'eval_loss': 0.25742781162261963, 'eval_accuracy': 0.9143501126972201, 'eval_precision': 0.908957415565345, 'eval_recall': 0.9225037257824144, 'eval_f1': 0.9156804733727811, 'eval_runtime': 7.2263, 'eval_samples_per_second': 184.188, 'eval_steps_per_second': 11.624, 'epoch': 1.0}


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(



=== RESULTADOS DE ERRO ===
Total: 1331
 Acertos: 1217
 Erros: 114
  ↳ Falsos Positivos (prevê ofensa mas não é): 62
  ↳ Falsos Negativos (não detectou ofensa real): 52

===  FALSOS POSITIVOS ===
• Cada dia se queimando mais
• É a treva.
• Isto mesmo. Esses filhos do Sr. Presidente deveriam ficar mais falados. Vão colocar tudo a perder. Uma lástima.
• Jesus, é cada uma desse cidadão. É de assustar.
• SENSACIONAL É BEM PÔR AI MESMOOO, ELE ESTÁ PREOCUPADO PORQUÊ SERA, QUEM Ñ DEVE Ñ TEME, SIMPLES ASSIM!!!
• Aê num aguento!
• Aro
• Dupla imbatível nas argumentações inteligentes contra esse governo desgovernado que estamos pagando....aff
• E nós que podemos ter perdido uma Greta Thunberg cruelmente assasinada por witizelassassino witizelgenocida. Ághata nossa Menina Maravilha tinha apenas 8 anos seu discurso era apenas a boneca Mônica, mas quem poderá afirmar ao contrário que aos 16 anos trocaria Mônica pela bandeira de um mundo mais justo e sustentável.
• Essa menina sozinha ta fazendo mai

In [ ]:
model_name = "/kaggle/input/bertimbau-tcc-model/bert-base-portuguese-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForMaskedLM.from_pretrained(model_name)
model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"✅ Modelo carregado no dispositivo: {device}")

def mlm_augment(text, idx=None, prob=0.20, n_aug=5):
    """
    Gera frases aumentadas via Masked Language Modeling (BERTimbau)
    e associa cada uma ao índice original do texto.

    Args:
        text (str): Texto original.
        idx (int, optional): Índice da frase original no dataset.
        prob (float): Probabilidade de mascarar um token.
        n_aug (int): Quantas versões aumentadas gerar.

    Returns:
        list[dict]: Lista com {"orig_idx", "text_original", "text_augmentada"}
    """
    if not isinstance(text, str) or text.strip() == "":
        return []

    tokens = tokenizer.tokenize(text)
    augmented_samples = []

    for _ in range(n_aug):
        masked_tokens = tokens.copy()
        for i in range(len(masked_tokens)):
            if random.random() < prob and masked_tokens[i].isalpha():
                masked_tokens[i] = tokenizer.mask_token

        masked_text = tokenizer.convert_tokens_to_string(masked_tokens)
        inputs = tokenizer(masked_text, return_tensors='pt').to(device)

        with torch.no_grad():
            outputs = model(**inputs)
            predictions = outputs.logits

        predicted_indices = torch.argmax(predictions, dim=-1)
        if predicted_indices.dim() == 0:
            predicted_indices = predicted_indices.unsqueeze(0)

        predicted_tokens = tokenizer.convert_ids_to_tokens(predicted_indices[0].tolist())

        result_tokens = [
            predicted_tokens[i] if masked_tokens[i] == tokenizer.mask_token else masked_tokens[i]
            for i in range(len(masked_tokens))
        ]

        new_text = tokenizer.convert_tokens_to_string(result_tokens)
        augmented_samples.append({
            "orig_idx": idx,
            "text_original": text,
            "text_augmentada": new_text
        })

    return augmented_samples


# ============================================================
# Carregar dataset
# ============================================================
train_path = "/kaggle/input/treinamento-e-teste/train_df.csv"
train_df = pd.read_csv(train_path)

augmented_rows = []

# Itera sobre cada linha do treino
for idx, row in tqdm(train_df.iterrows(), total=len(train_df)):
    # Gera novas versões da frase
    new_samples = mlm_augment(
        text=row["text"],
        idx=idx,           # índice original da linha no treino
        prob=0.20,         # 15% das palavras mascaradas
        n_aug=5            # duas versões aumentadas por frase
    )

    # Adiciona resultados ao dataset final
    for sample in new_samples:
        new_text = sample["text_augmentada"]

        augmented_rows.append({
            "orig_idx": sample["orig_idx"],
            "text_original": sample["text_original"],
            "text_augmentada": new_text,
            "class": row["class"]
        })

# Cria DataFrame com os textos aumentados
df_aug_2 = pd.DataFrame(augmented_rows)

✅ Modelo carregado no dispositivo: cuda


100%|██████████| 5322/5322 [03:49<00:00, 23.21it/s]


In [ ]:
df_aug_2= df_aug_2[["text_augmentada", "class"]].rename(columns={"text_augmentada": "text"})

print(df_aug_2.shape)
df_aug_2.head()

(26610, 2)


,text,class
0,Parabéns meu presidente,0
1,Parabéns meu meu,0
2,Parabénséns presidente,0
3,Parabéns meu meu,0
4,Parabénséns presidente,0


In [ ]:
train_path = "/kaggle/input/treinamento-e-teste/train_df.csv"
test_path = "/kaggle/input/treinamento-e-teste/test_df.csv"

# Carregar datasets
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

train_final = pd.concat([train_df, df_aug_2], ignore_index=True)

#  Renomear 'class' → 'labels'
train_final = train_final.rename(columns={"class": "labels"})
test_df = test_df.rename(columns={"class": "labels"})


train_final = train_final[~train_final["text"].str.contains(r"\[UNK\]", regex=True, na=False)]
train_final = train_final.drop_duplicates(subset=["text"], keep="first").reset_index(drop=True)

#  Remover nulos e garantir tipo string
for name, df_ in {"train": train_final, "test": test_df}.items():
    df_.dropna(subset=["text", "labels"], inplace=True)
    df_["text"] = df_["text"].astype(str)
    df_.loc[df_["text"].str.strip() == "", "text"] = np.nan
    df_.dropna(subset=["text"], inplace=True)
    print(f" {name}: {len(df_)} linhas")

#  Tokenização
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=128
    )

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(p.label_ids, preds, average="binary")
    acc = accuracy_score(p.label_ids, preds)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

train_dataset = Dataset.from_pandas(train_final)
test_dataset = Dataset.from_pandas(test_df)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# ============================================================
# 9 Treinamento e avaliação
# ============================================================
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="no",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()
metrics = trainer.evaluate()
print(metrics)

# ============================================================
# 9. Análise de Erros
# ============================================================
preds_output = trainer.predict(test_dataset)
preds = np.argmax(preds_output.predictions, axis=1)

result_df = test_df.copy()
result_df["preds"] = preds

total = len(result_df)
acertos = (result_df["labels"] == result_df["preds"]).sum()
erros = total - acertos
fp = ((result_df["preds"] == 1) & (result_df["labels"] == 0)).sum()
fn = ((result_df["preds"] == 0) & (result_df["labels"] == 1)).sum()

print(f"\n=== RESULTADOS DE ERRO ===")
print(f"Total: {total}")
print(f" Acertos: {acertos}")
print(f" Erros: {erros}")
print(f"  ↳ Falsos Positivos (prevê ofensa mas não é): {fp}")
print(f"  ↳ Falsos Negativos (não detectou ofensa real): {fn}")

# Mostrar exemplos
falsos_positivos = result_df[(result_df["preds"] == 1) & (result_df["labels"] == 0)]
falsos_negativos = result_df[(result_df["preds"] == 0) & (result_df["labels"] == 1)]

print("\n===  FALSOS POSITIVOS ===")
for t in falsos_positivos["text"].head(30):
    print("•", t)

print("\n===  FALSOS NEGATIVOS ===")
for t in falsos_negativos["text"].head(30):
    print("•", t)

 train: 24088 linhas
 test: 1331 linhas


Map:   0%|          | 0/24088 [00:00<?, ? examples/s]

Map:   0%|          | 0/1331 [00:00<?, ? examples/s]

/tmp/ipykernel_37/3267608434.py:68: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


{'loss': 0.5505, 'grad_norm': 1.7914618253707886, 'learning_rate': 1.9349269588313415e-05, 'epoch': 0.033200531208499334}
{'loss': 0.3775, 'grad_norm': 9.333806037902832, 'learning_rate': 1.868525896414343e-05, 'epoch': 0.06640106241699867}
{'loss': 0.3199, 'grad_norm': 10.904051780700684, 'learning_rate': 1.8021248339973442e-05, 'epoch': 0.099601593625498}
{'loss': 0.2919, 'grad_norm': 16.351192474365234, 'learning_rate': 1.7357237715803456e-05, 'epoch': 0.13280212483399734}
{'loss': 0.2494, 'grad_norm': 2.6490745544433594, 'learning_rate': 1.669322709163347e-05, 'epoch': 0.16600265604249667}
{'loss': 0.2621, 'grad_norm': 1.1895049810409546, 'learning_rate': 1.602921646746348e-05, 'epoch': 0.199203187250996}
{'loss': 0.2262, 'grad_norm': 15.565313339233398, 'learning_rate': 1.5365205843293494e-05, 'epoch': 0.23240371845949534}
{'loss': 0.2849, 'grad_norm': 11.781869888305664, 'learning_rate': 1.4701195219123507e-05, 'epoch': 0.2656042496679947}
{'loss': 0.2267, 'grad_norm': 12.0898742

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


{'eval_loss': 0.29196697473526, 'eval_accuracy': 0.9166040570999249, 'eval_precision': 0.9154302670623146, 'eval_recall': 0.9195230998509687, 'eval_f1': 0.9174721189591077, 'eval_runtime': 7.5798, 'eval_samples_per_second': 175.598, 'eval_steps_per_second': 11.082, 'epoch': 1.0}
{'eval_loss': 0.29196697473526, 'eval_accuracy': 0.9166040570999249, 'eval_precision': 0.9154302670623146, 'eval_recall': 0.9195230998509687, 'eval_f1': 0.9174721189591077, 'eval_runtime': 7.5798, 'eval_samples_per_second': 175.598, 'eval_steps_per_second': 11.082, 'epoch': 1.0}


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(



=== RESULTADOS DE ERRO ===
Total: 1331
 Acertos: 1220
 Erros: 111
  ↳ Falsos Positivos (prevê ofensa mas não é): 57
  ↳ Falsos Negativos (não detectou ofensa real): 54

===  FALSOS POSITIVOS ===
• Cada dia se queimando mais
• É a treva.
• Isto mesmo. Esses filhos do Sr. Presidente deveriam ficar mais falados. Vão colocar tudo a perder. Uma lástima.
• Jesus, é cada uma desse cidadão. É de assustar.
• SENSACIONAL É BEM PÔR AI MESMOOO, ELE ESTÁ PREOCUPADO PORQUÊ SERA, QUEM Ñ DEVE Ñ TEME, SIMPLES ASSIM!!!
• O Brasil lutando para sair da Grota e alguns preocupado com a tal de Greta. Vamos ficar até quando nessa?
• Aro
• A tá ,a esquerda pode falar que vai incendiar o país e ninguém fala nada.
• E nós que podemos ter perdido uma Greta Thunberg cruelmente assasinada por witizelassassino witizelgenocida. Ághata nossa Menina Maravilha tinha apenas 8 anos seu discurso era apenas a boneca Mônica, mas quem poderá afirmar ao contrário que aos 16 anos trocaria Mônica pela bandeira de um mundo mais 

In [ ]:
import torch
import random
import re
import nltk
from nltk.corpus import stopwords
from sentence_transformers import SentenceTransformer, util

# Garantir que stopwords estão disponíveis
nltk.download("stopwords")
stopwords_pt = set(stopwords.words("portuguese"))

# Carrega modelo SBERT (para medir similaridade semântica)
sbert = SentenceTransformer('/kaggle/input/miniml/paraphrase-multilingual-MiniLM-L12-v2')


def debug_mlm_augment_conditional_one_token_dataset(
    text,
    tokenizer,
    model,
    device,
    n_aug=10,
    top_k=10,
    max_tries=20,
    min_similarity=0.7
):
    """
    Versão com debug detalhado do gerador de augmentations via MLM.
    Mostra cada tentativa, similaridade e motivo de rejeição.
    """

    stop_words = set(stopwords.words("portuguese"))
    augmented_samples = []

    print("=" * 100)
    print(f"📝 Texto original: {text}")
    print("-" * 100)

    # Limpeza básica
    text = re.sub(r"[‘’]", "'", text)
    text = re.sub(r"[“”]", '"', text)

    tokens = tokenizer.tokenize(text)
    if len(tokens) < 3:
        print(" Frase muito curta (menos de 3 tokens).")
        print("=" * 100)
        return []

    def is_full_word(tokens, idx):
        """True se token for palavra completa, não subpalavra."""
        if tokens[idx].startswith("##"):
            return False
        if idx + 1 < len(tokens) and tokens[idx + 1].startswith("##"):
            return False
        return True

    candidate_indices = [
        i for i, t in enumerate(tokens)
        if t.isalpha() and is_full_word(tokens, i) and t.lower() not in stop_words
    ]

    if not candidate_indices:
        print(" Nenhuma palavra candidata para mascarar.")
        print("=" * 100)
        return []

    print(f" Tokens candidatos ({len(candidate_indices)}): {[tokens[i] for i in candidate_indices]}")

    # Embedding do texto original
    orig_emb = sbert.encode(text, convert_to_tensor=True, show_progress_bar=False)

    tries = 0
    total_attempts = 0
    while len(augmented_samples) < n_aug and tries < max_tries:
        tries += 1

        mask_idx = random.choice(candidate_indices)
        masked_tokens = tokens.copy()
        masked_tokens[mask_idx] = tokenizer.mask_token
        masked_text = tokenizer.convert_tokens_to_string(masked_tokens)

        inputs = tokenizer(masked_text, return_tensors="pt").to(device)
        mask_positions = (inputs["input_ids"] == tokenizer.mask_token_id).nonzero(as_tuple=False)
        if mask_positions.size(0) == 0:
            print(" Nenhum [MASK] encontrado após tokenização.")
            continue
        mask_pos = mask_positions[0, 1].item()

        with torch.no_grad():
            outputs = model(**inputs)
        logits = outputs.logits[0, mask_pos]
        top_k_ids = torch.topk(logits, k=min(top_k, logits.size(0))).indices.tolist()
        random.shuffle(top_k_ids)

        for chosen_id in top_k_ids:
            total_attempts += 1
            new_token = tokenizer.convert_ids_to_tokens([chosen_id])[0]
            if new_token == "[UNK]" or new_token.startswith("##"):
                print(f" Rejeitado: token inválido ({new_token})")
                continue

            new_tokens = tokens.copy()
            new_tokens[mask_idx] = new_token
            new_text = tokenizer.convert_tokens_to_string(new_tokens).strip()

            if "[UNK]" in new_text:
                print(f" Rejeitado: contém [UNK] → {new_text}")
                continue

            if new_text.lower() == text.strip().lower():
                print(f" Rejeitado: igual ao original → {new_text}")
                continue

            # Calcula similaridade
            new_emb = sbert.encode(new_text, convert_to_tensor=True, show_progress_bar=False)
            sim = util.cos_sim(orig_emb, new_emb).item()

            if sim < min_similarity:
                print(f" Rejeitado (sim={sim:.3f} < {min_similarity}) → {new_text}")
                continue

            if new_text in augmented_samples:
                print(f" Já gerado → {new_text}")
                continue

            augmented_samples.append(new_text)
            print(f"✅ Aceito (sim={sim:.3f}) → {new_text}")

            if len(augmented_samples) >= n_aug:
                break

    print("-" * 100)
    print(f" Total aceitas: {len(augmented_samples)} / {n_aug} (em {total_attempts} tentativas)")
    print("=" * 100)
    return augmented_samples


# Exemplo de uso com seu dataset
import pandas as pd

train_path = "/kaggle/input/treinamento-e-teste/train_df.csv"
train_df = pd.read_csv(train_path)

for i, row in train_df.head(50).iterrows():
    novas = debug_mlm_augment_conditional_one_token_dataset(
        text=row["text"],
        tokenizer=tokenizer,
        model=model,
        device=device,
        n_aug=10,
        top_k=10,
        max_tries=20,
        min_similarity=0.7,
    )

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


📝 Texto original: Parabéns meu presidente
----------------------------------------------------------------------------------------------------
 Tokens candidatos (1): ['presidente']
✅ Aceito (sim=0.715) → Parabéns meu !
 Rejeitado (sim=0.688 < 0.7) → Parabéns meu filho
 Rejeitado (sim=0.655 < 0.7) → Parabéns meu amor
 Rejeitado (sim=0.664 < 0.7) → Parabéns meu pai
 Rejeitado (sim=0.680 < 0.7) → Parabéns meu neto
✅ Aceito (sim=0.725) → Parabéns meu caro
 Rejeitado (sim=0.671 < 0.7) → Parabéns meu amigo
✅ Aceito (sim=0.743) → Parabéns meu .
 Rejeitado (sim=0.591 < 0.7) → Parabéns meu coração
 Rejeitado (sim=0.624 < 0.7) → Parabéns meu irmão
 Rejeitado (sim=0.688 < 0.7) → Parabéns meu filho
 Já gerado → Parabéns meu .
 Rejeitado (sim=0.655 < 0.7) → Parabéns meu amor
 Já gerado → Parabéns meu caro
 Rejeitado (sim=0.664 < 0.7) → Parabéns meu pai
 Rejeitado (sim=0.671 < 0.7) → Parabéns meu amigo
 Já gerado → Parabéns meu !
 Rejeitado (sim=0.591 < 0.7) → Parabéns meu coração
 Rejeitado (sim=0

In [ ]:
import torch
import random
import re
import nltk
import pandas as pd
from nltk.corpus import stopwords
from sentence_transformers import SentenceTransformer, util

# ============================================================
# 1. Preparação
# ============================================================
nltk.download("stopwords")
stopwords_pt = set(stopwords.words("portuguese"))

# Carrega modelo SBERT
sbert = SentenceTransformer('/kaggle/input/miniml/paraphrase-multilingual-MiniLM-L12-v2')

# ============================================================
# 2. Função de data augmentation (sem prints)
# ============================================================
def mlm_augment_clean(
    text,
    tokenizer,
    model,
    device,
    n_aug=10,
    top_k=10,
    max_tries=20,
    min_similarity=0.7,
    max_similarity=1.0
):
    stop_words = set(stopwords.words("portuguese"))
    augmented_samples = []

    text = re.sub(r"[‘’]", "'", text)
    text = re.sub(r"[“”]", '"', text)

    tokens = tokenizer.tokenize(text)
    if len(tokens) < 3:
        return []

    def is_full_word(tokens, idx):
        if tokens[idx].startswith("##"):
            return False
        if idx + 1 < len(tokens) and tokens[idx + 1].startswith("##"):
            return False
        return True

    candidate_indices = [
        i for i, t in enumerate(tokens)
        if t.isalpha() and is_full_word(tokens, i) and t.lower() not in stop_words
    ]
    if not candidate_indices:
        return []

    orig_emb = sbert.encode(text, convert_to_tensor=True, show_progress_bar=False)

    tries = 0
    while len(augmented_samples) < n_aug and tries < max_tries:
        tries += 1
        mask_idx = random.choice(candidate_indices)
        masked_tokens = tokens.copy()
        masked_tokens[mask_idx] = tokenizer.mask_token
        masked_text = tokenizer.convert_tokens_to_string(masked_tokens)

        inputs = tokenizer(masked_text, return_tensors="pt").to(device)
        mask_positions = (inputs["input_ids"] == tokenizer.mask_token_id).nonzero(as_tuple=False)
        if mask_positions.size(0) == 0:
            continue
        mask_pos = mask_positions[0, 1].item()

        with torch.no_grad():
            outputs = model(**inputs)
        logits = outputs.logits[0, mask_pos]
        top_k_ids = torch.topk(logits, k=min(top_k, logits.size(0))).indices.tolist()
        random.shuffle(top_k_ids)

        for chosen_id in top_k_ids:
            new_token = tokenizer.convert_ids_to_tokens([chosen_id])[0]
            if new_token == "[UNK]" or new_token.startswith("##"):
                continue

            new_tokens = tokens.copy()
            new_tokens[mask_idx] = new_token
            new_text = tokenizer.convert_tokens_to_string(new_tokens).strip()

            if "[UNK]" in new_text or new_text.lower() == text.lower():
                continue

            new_emb = sbert.encode(new_text, convert_to_tensor=True, show_progress_bar=False)
            sim = util.cos_sim(orig_emb, new_emb).item()

            if not (min_similarity <= sim <= max_similarity):
                continue

            if new_text in augmented_samples:
                continue

            augmented_samples.append(new_text)

            if len(augmented_samples) >= n_aug:
                break

    return augmented_samples

# ============================================================
# 3. Geração dos 5 datasets
# ============================================================
intervalos = {
    "sim_0.65_1.00": (0.65, 1.00),
    "sim_0.70_1.00": (0.70, 1.00),
    "sim_0.65_0.90": (0.65, 0.90),
    "sim_0.70_0.90": (0.70, 0.90),
    "sim_0.70_0.85": (0.70, 0.85),
}

train_path = "/kaggle/input/treinamento-e-teste/train_df.csv"
train_df = pd.read_csv(train_path)

# Gera datasets
for nome, (min_sim, max_sim) in intervalos.items():
    print(f" Gerando dataset {nome} (intervalo {min_sim}-{max_sim})")

    aug_texts, aug_labels = [], []
    for i, row in train_df.iterrows():
        novas = mlm_augment_clean(
            text=row["text"],
            tokenizer=tokenizer,
            model=model,
            device=device,
            n_aug=10,
            top_k=10,
            max_tries=20,
            min_similarity=min_sim,
            max_similarity=max_sim,
        )
        for n in novas:
            aug_texts.append(n)
            aug_labels.append(row["class"])

    df_aug = pd.DataFrame({"text": aug_texts, "class": aug_labels})
    df_final = pd.concat([train_df, df_aug], ignore_index=True)
    df_final.to_csv(f"/kaggle/working/train_aug_{nome}.csv", index=False)

    print(f" Salvo: /kaggle/working/train_aug_{nome}.csv — {len(df_final)} linhas totais\n")

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


 Gerando dataset sim_0.65_1.00 (intervalo 0.65-1.0)
 Salvo: /kaggle/working/train_aug_sim_0.65_1.00.csv — 43250 linhas totais

 Gerando dataset sim_0.70_1.00 (intervalo 0.7-1.0)
 Salvo: /kaggle/working/train_aug_sim_0.70_1.00.csv — 42459 linhas totais

 Gerando dataset sim_0.65_0.90 (intervalo 0.65-0.9)
 Salvo: /kaggle/working/train_aug_sim_0.65_0.90.csv — 32193 linhas totais

 Gerando dataset sim_0.70_0.90 (intervalo 0.7-0.9)
 Salvo: /kaggle/working/train_aug_sim_0.70_0.90.csv — 30551 linhas totais

 Gerando dataset sim_0.70_0.85 (intervalo 0.7-0.85)
 Salvo: /kaggle/working/train_aug_sim_0.70_0.85.csv — 22515 linhas totais



In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch
import os

# ============================================================
# 1) Configurações
# ============================================================
model_name = "/kaggle/input/bertimbau-tcc-model/bert-base-portuguese-cased"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained(model_name)

base_input = "/kaggle/input/treinamentoad"
datasets_paths = {
    "sim_0.65_1.00": "/kaggle/working/train_aug_sim_0.65_1.00.csv",
    "sim_0.70_1.00": "/kaggle/working/train_aug_sim_0.70_1.00.csv",
    "sim_0.65_0.90": "/kaggle/working/train_aug_sim_0.65_0.90.csv",
    "sim_0.70_0.90": "/kaggle/working/train_aug_sim_0.70_0.90.csv",
    "sim_0.70_0.85": "/kaggle/working/train_aug_sim_0.70_0.85.csv",
}

test_path = "/kaggle/input/treinamento-e-teste/test_df.csv"
test_df = pd.read_csv(test_path)
test_df = test_df.rename(columns={"class": "labels"})

# ============================================================
# 2) Funções utilitárias
# ============================================================
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(p.label_ids, preds, average="binary")
    acc = accuracy_score(p.label_ids, preds)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# ============================================================
# 3) Avaliar cada dataset
# ============================================================
results = []

for name, path in datasets_paths.items():
    print(f"\n Treinando modelo para dataset: {name}")

    # Carregar dataset
    train_df = pd.read_csv(path)
    if "class" in train_df.columns:
        train_df = train_df.rename(columns={"class": "labels"})

    # Limpeza
    train_df = train_df.drop_duplicates(subset=["text"], keep="first").dropna(subset=["text", "labels"]).reset_index(drop=True)
    train_df["text"] = train_df["text"].astype(str)

    # Tokenização
    train_dataset = Dataset.from_pandas(train_df)
    test_dataset = Dataset.from_pandas(test_df)
    train_dataset = train_dataset.map(tokenize_function, batched=True)
    test_dataset = test_dataset.map(tokenize_function, batched=True)
    train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
    test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

    # Modelo e treino
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2).to(device)

    training_args = TrainingArguments(
        output_dir=f"./results_{name}",
        eval_strategy="epoch",
        save_strategy="no",
        learning_rate=2e-5,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=1,
        weight_decay=0.01,
        logging_dir=f"./logs_{name}",
        logging_steps=100,
        report_to="none"  # evita logs no WandB
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    trainer.train()
    metrics = trainer.evaluate()

    results.append({
        "dataset": name,
        "accuracy": metrics["eval_accuracy"],
        "precision": metrics["eval_precision"],
        "recall": metrics["eval_recall"],
        "f1": metrics["eval_f1"]
    })

# ============================================================
# 4) Resumo comparativo
# ============================================================
results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by="f1", ascending=False).reset_index(drop=True)

print("\n RESULTADOS FINAIS")
print(results_df)

# Salvar resultados
results_df.to_csv("/kaggle/working/bert_results_comparativo.csv", index=False)
print("\n Resultados salvos em /kaggle/working/bert_results_comparativo.csv")


 Treinando modelo para dataset: sim_0.65_1.00


Map:   0%|          | 0/43042 [00:00<?, ? examples/s]

Map:   0%|          | 0/1331 [00:00<?, ? examples/s]

/tmp/ipykernel_37/856745935.py:84: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


{'loss': 0.4966, 'grad_norm': 9.776448249816895, 'learning_rate': 1.9264214046822744e-05, 'epoch': 0.03716090672612412}
{'loss': 0.2716, 'grad_norm': 9.232985496520996, 'learning_rate': 1.8520995912300262e-05, 'epoch': 0.07432181345224824}
{'loss': 0.2242, 'grad_norm': 14.672833442687988, 'learning_rate': 1.7777777777777777e-05, 'epoch': 0.11148272017837235}
{'loss': 0.2181, 'grad_norm': 9.53431224822998, 'learning_rate': 1.7034559643255298e-05, 'epoch': 0.14864362690449648}
{'loss': 0.2009, 'grad_norm': 11.961630821228027, 'learning_rate': 1.6291341508732813e-05, 'epoch': 0.18580453363062058}
{'loss': 0.1617, 'grad_norm': 1.8552415370941162, 'learning_rate': 1.554812337421033e-05, 'epoch': 0.2229654403567447}
{'loss': 0.1348, 'grad_norm': 7.390620231628418, 'learning_rate': 1.480490523968785e-05, 'epoch': 0.2601263470828688}
{'loss': 0.1331, 'grad_norm': 0.32121601700782776, 'learning_rate': 1.4061687105165367e-05, 'epoch': 0.29728725380899296}
{'loss': 0.1296, 'grad_norm': 19.9239177

...

Map:   0%|          | 0/1331 [00:00<?, ? examples/s]

/tmp/ipykernel_37/856745935.py:84: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


{'loss': 0.4582, 'grad_norm': 8.00192642211914, 'learning_rate': 1.858672376873662e-05, 'epoch': 0.07137758743754462}
{'loss': 0.2877, 'grad_norm': 5.8521952629089355, 'learning_rate': 1.7159172019985725e-05, 'epoch': 0.14275517487508924}
{'loss': 0.2215, 'grad_norm': 0.45973750948905945, 'learning_rate': 1.5731620271234832e-05, 'epoch': 0.21413276231263384}
{'loss': 0.2057, 'grad_norm': 11.597918510437012, 'learning_rate': 1.430406852248394e-05, 'epoch': 0.28551034975017847}
{'loss': 0.1933, 'grad_norm': 6.175741195678711, 'learning_rate': 1.287651677373305e-05, 'epoch': 0.35688793718772305}
{'loss': 0.1831, 'grad_norm': 13.074370384216309, 'learning_rate': 1.1448965024982157e-05, 'epoch': 0.4282655246252677}
{'loss': 0.1498, 'grad_norm': 16.43013572692871, 'learning_rate': 1.0021413276231265e-05, 'epoch': 0.49964311206281226}
{'loss': 0.1335, 'grad_norm': 0.10346709191799164, 'learning_rate': 8.593861527480372e-06, 'epoch': 0.5710206995003569}
{'loss': 0.0973, 'grad_norm': 0.07456655

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


{'eval_loss': 0.39392223954200745, 'eval_accuracy': 0.9075882794891059, 'eval_precision': 0.9053254437869822, 'eval_recall': 0.9120715350223547, 'eval_f1': 0.9086859688195991, 'eval_runtime': 7.5033, 'eval_samples_per_second': 177.39, 'eval_steps_per_second': 11.195, 'epoch': 1.0}

 RESULTADOS FINAIS
         dataset  accuracy  precision    recall        f1
0  sim_0.65_0.90  0.913599   0.916168  0.912072  0.914115
1  sim_0.65_1.00  0.909842   0.899855  0.923994  0.911765
2  sim_0.70_1.00  0.909842   0.902190  0.921013  0.911504
3  sim_0.70_0.90  0.907588   0.904130  0.913562  0.908821
4  sim_0.70_0.85  0.907588   0.905325  0.912072  0.908686

 Resultados salvos em /kaggle/working/bert_results_comparativo.csv
